In [16]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from konlpy.tag import Okt
from gensim.models import Word2Vec
from scipy.spatial import distance
from konlpy.tag import Mecab

## 전처리

In [25]:
raw_df = pd.read_csv('/Users/jungyulyang/programming/Project_ThematicInvest/Data/data_for_use/theme_contents.csv',
                     index_col=0, encoding = 'utf-8')
raw_df.dropna(inplace=True)

In [26]:
raw_df

,theme,contents
0,2차전지,신종 코로나바이러스 감염증(코로나19) 사태로 인한 급락장에서 10대 그룹 시가총...
1,2차전지,2차전지·디스플레이 공정 장비 전문기업 나인테크가 기업인수목적회사(SPAC·스팩) ...
2,2차전지,2차전지·디스플레이 공정 장비 전문기업 나인테크가 교보7호스팩과 합병을 통한 코스...
3,2차전지,'울산형 뉴딜사업' 추진 등 코로나19 이후 지역경제 살리기에 나선 울산에 반가운...
4,2차전지,코스닥 2차 전지 소재 기업 에코프로비엠이 압도적 기술력을 바탕으로 가파른 성장 ...
...,...,...
33595,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...
33596,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...
33597,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...
33598,희토류,희귀금속(희토류 등) 테마 주요 종목 등락률씽크풀 로보알고리즘 RASSI의 분석에...


In [27]:
raw_df = raw_df[['theme','contents']]
raw_df['theme'] = raw_df['Article'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
raw_df.head()

,theme,contents
0,2차전지,신종 코로나바이러스 감염증코로나 사태로 인한 급락장에서 대 그룹 시가총액 반등을 ...
1,2차전지,차전지디스플레이 공정 장비 전문기업 나인테크가 기업인수목적회사스팩 합병을 통해 코스...
2,2차전지,차전지디스플레이 공정 장비 전문기업 나인테크가 교보호스팩과 합병을 통한 코스닥 상...
3,2차전지,울산형 뉴딜사업 추진 등 코로나 이후 지역경제 살리기에 나선 울산에 반가운 소식이...
4,2차전지,코스닥 차 전지 소재 기업 에코프로비엠이 압도적 기술력을 바탕으로 가파른 성장 추...


In [29]:
# tokenize data

tokenizer = Mecab()

raw_tokenized_data = []
for content in tqdm(raw_df['contents']):
    token = tokenizer.morphs(content)
    stop_words = ['으로', '로도', '지만', '에서', '려는', '하다']
    for word in token:
        if len(word) == 1:
            stop_words.append(word)
    final_tokens = [word for word in token if word not in stop_words]
    raw_tokenized_data.append(final_tokens)

100%|██████████| 33600/33600 [01:14<00:00, 453.02it/s]


In [30]:
# make train_data

train_data = raw_df.copy()
train_data['tokens'] = pd.Series(raw_tokenized_data)

In [31]:
train_data.head()

,theme,contents,tokens
0,2차전지,신종 코로나바이러스 감염증코로나 사태로 인한 급락장에서 대 그룹 시가총액 반등을 ...,"[신종, 코로나, 바이러스, 감염증, 코로나, 사태, 인한, 급락, 그룹, 시가총액..."
1,2차전지,차전지디스플레이 공정 장비 전문기업 나인테크가 기업인수목적회사스팩 합병을 통해 코스...,"[전지, 디스플레이, 공정, 장비, 전문, 기업, 나인, 테크, 기업, 수목, 회사..."
2,2차전지,차전지디스플레이 공정 장비 전문기업 나인테크가 교보호스팩과 합병을 통한 코스닥 상...,"[전지, 디스플레이, 공정, 장비, 전문, 기업, 나인, 테크, 교보, 호스, 합병..."
3,2차전지,울산형 뉴딜사업 추진 등 코로나 이후 지역경제 살리기에 나선 울산에 반가운 소식이...,"[울산, 뉴딜, 사업, 추진, 코로나, 이후, 지역, 경제, 살리, 나선, 울산, ..."
4,2차전지,코스닥 차 전지 소재 기업 에코프로비엠이 압도적 기술력을 바탕으로 가파른 성장 추...,"[코스닥, 전지, 소재, 기업, 프로, 비엠, 압도, 기술력, 바탕, 가파른, 성장..."


## Model Train

In [32]:
news_tokens = train_data['tokens'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8

model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)

In [33]:
model.wv.vectors.shape

(38967, 300)

In [34]:
print(model.wv.most_similar("아이폰"))

[('애플', 0.743939995765686), ('아이패드', 0.7104508876800537), ('안드로이드', 0.6580507755279541), ('맥북', 0.6435776948928833), ('블랙베리', 0.6187098622322083), ('스냅드래곤', 0.6039987206459045), ('노치', 0.5979969501495361), ('버즈', 0.5701177716255188), ('갤럭시', 0.5671680569648743), ('화면', 0.5525581240653992)]


In [38]:
# sentence vectors without normalization

def without_normal(tokens):
    vectors = []
    for token in tokens:
        init_v = np.array([0.0]*v_dimension)
        for word in token:
            word_vectors = model.wv
            if word in word_vectors.vocab:
                v = model.wv[word]
                init_v = init_v + v
        vectors.append(init_v)
    
    frame = { 'themes': train_data['theme'].tolist(), 'vectors': vectors }
    result = pd.DataFrame(frame) 
    
    return result

In [39]:
# theme vectors without normalization

def theme_without_normal(news_df):
    theme_list = []
    vector_list = []
    for theme in news_df['themes'].unique():
        temp_df = news_df.loc[news_df['themes'] == theme]
        add_v = np.array([0.0]*v_dimension)
        for vec in temp_df['vectors']:
            add_v  = add_v + vec
        theme_list.append(theme)
        vector_list.append(add_v)
        
    frame = { 'themes': theme_list, 'vectors': vector_list }
    result = pd.DataFrame(frame)
    
    return result

In [40]:
news_vectors_df = without_normal(train_data['tokens'])

In [42]:
news_vectors_df.head()

,themes,vectors
0,2차전지,"[32.52144870487973, -25.973187564406544, -6.49..."
1,2차전지,"[27.493043491616845, -67.19967107940465, -47.6..."
2,2차전지,"[29.5968237798661, -47.93156994599849, -39.006..."
3,2차전지,"[-111.57007592823356, -112.60198955237865, 14...."
4,2차전지,"[-7.244672171305865, -67.69402845273726, 67.65..."


In [18]:
theme_vectors_df = theme_without_normal(news_vectors_df)

In [41]:
theme_vectors_df.head()

NameError: name 'theme_vectors_df' is not defined

In [26]:
theme_vectors_df.to_csv('theme_vectors_xnorm')

## Test

In [20]:
# news vectors without normalization

def vectorize_without_normal(news):
    news_words = news.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    token = tokenizer.morphs(news)
    stop_words = ['으로', '로도', '지만', '에서', '려는', '하다']
    for word in token:
        if len(word) == 1:
            stop_words.append(word)
    final_tokens = [word for word in token if word not in stop_words]
    init_v = np.array([0.0]*v_dimension)
    for word in final_tokens:
        word_vectors = model.wv
        if word in word_vectors.vocab:
            v = model.wv[word]
            init_v = init_v + v
    return init_v

In [21]:
# test

def test(news):
    news_vec = vectorize_without_normal(news)
    result = []
    for theme in theme_vectors_df['vectors']:
        cosine = 1 - distance.cosine(theme, news_vec)
        result.append(cosine)
    new_df = pd.DataFrame(data=np.zeros([282,2]), columns=['Theme', 'Result'])
    new_df['Theme'] = theme_vectors_df['themes']
    new_df['Result'] = result

    new_df.sort_values('Result', ascending=False, inplace=True)
    return new_df
    

In [22]:
test_news = "미국의 최대 새해맞이 행사죠, '뉴 이어스 로킹 이브'에 방탄소년단이 출연해 월드 스타로서의 면모를 과시했습니다. 지난해 빌보드 순위와 월드 투어에서 괄목할 만한 성장을 거둔 방탄소년단, 올해도 한류를 이끌 것으로 기대되고 있습니다. 김혜은 기자입니다. 방탄소년단, BTS는 미국 뉴욕의 심장인 타임스퀘어에서 새해를 맞았습니다. 미국의 최대 새해맞이 행사인 '뉴 이어스 로킹 이브'에 참가한 것입니다. 포스트 말론, 샘 헌트 등 세계적인 가수들과 어깨를 나란히 하며 미국 안방에 스며들었습니다. [리안나 제이콥슨 / BTS 팬 : BTS가 사랑받고 있다는 것, 그들이 놀랍고 재능 있고 매 순간 가치 있다는 것을 알려줘야 합니다.] BTS는 지난해 '빌보드 200' 1위와 아메리칸 뮤직 어워즈 3관왕 등 괄목할 기록을 낳았습니다. 한국 가수 최초로 전 세계 스타디움 투어도 성공적으로 마무리했습니다. 투어의 대미를 장식한 서울 공연에서만 사흘 동안 무려 1조 원에 육박하는 경제 효과를 낳은 것으로 집계됐습니다. 공연으로 18만 명 넘는 외국인이 우리나라를 방문했는데, 평창동계올림픽 당시 외국인 방문객의 67%에 해당하는 수치입니다. BTS의 올해 행보에 더욱 기대가 쏠리는 대목입니다. 지난달 공개한 티저 사진으로 올 초 예정된 새 앨범과 투어에 대한 기대감이 올라갔습니다. [지민 / 방탄소년단 멤버 (지난달 MAMA 시상식) : 여러분들이 기대하시는 것보다 훨씬 더 좋은 앨범으로 저희가 여러분들에게 나타날 수 있을 것 같아요.] BTS 이후 '빌보드200' 1위를 차지한 슈퍼엠, 미국 프로그램에 자주 등장하는 몬스타엑스와 NCT 127 등 K팝의 지형은 갈수록 확대되고 있습니다. [김헌식 / 대중문화평론가 : 음악의 유통구조 자체가 유튜브를 포함한 SNS를 중심으로 확산하고 있기 때문에 여기에서 계속 추이를 따라가는 수준이라면 10년 이상까지도 가능할 수 있다는 거죠.] 새로운 10년이 시작되는 2020년, BTS가 미국 심장부인 뉴욕에서 새해를 시작한 것도, 한류의 새로운 10년에 대한 상징으로도 해석되고 있습니다."

In [23]:
result = test(test_news)

In [24]:
result.head()

,Theme,Result
7,기생충,0.777822
83,영화,0.663112
0,방탄소년단,0.656203
257,아기상어,0.647218
118,광고,0.593024
